# Using sklearn

In [110]:
import torch
from sklearn.linear_model import LinearRegression
from random import random, randint, choice
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


def randomize(factor):
    return choice([-1, 1]) * factor * random()


class Config:
    x_n = 3
    lr = 0.0001
    epochs = 20
    split_train = 0.8
    shuffle = True
    batch_size = 64
    device = torch.device("cpu")


config = Config()

In [111]:
seed_list = [random() for x in range(2 ** 18)]
x_n = config.x_n
coe = [0.1 * x for x in range(x_n)]

X = [[x + randomize(0.1) for y in range(x_n)] for x in seed_list]
y = [sum([x_i * coe_j + randomize(0.05) for x_i, coe_j in zip(x, coe)]) for x in X]

lr = LinearRegression(n_jobs=10)
lr.fit(X, y)

print(lr.score(X, y), lr.coef_, lr.intercept_)

lr.coef_

0.7545391831375537 [-0.00093028  0.10048545  0.20031834] 0.000258331786324284


array([-0.00093028,  0.10048545,  0.20031834])

# Using Neural Networks

In [112]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.cuda.device

torch.cuda.device

In [113]:
tensor_X = torch.tensor(X, dtype=torch.float32, device=config.device)
tensor_y = torch.tensor(y, dtype=torch.float32, device=config.device)

size_train = int(config.split_train * len(tensor_X))
size_test = len(tensor_X) - size_train
assert size_train + size_test == len(tensor_X)

dataset_train = TensorDataset(tensor_X[:size_train], tensor_y[:size_train])
dataset_test = TensorDataset(tensor_X[size_train:], tensor_y[size_train:])

loader_train = DataLoader(dataset=dataset_train, batch_size=config.batch_size)
loader_test = DataLoader(dataset=dataset_test, batch_size=config.batch_size)

In [114]:
class LinearModel(nn.Module):
    def __init__(self, in_features):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(in_features=in_features, out_features=1)

    def forward(self, x):
        return self.linear(x)


model = LinearModel(in_features=config.x_n)
model.to(device=config.device)


criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=config.lr)
loss_all = []

for epoch in tqdm(range(config.epochs)):
    loss_epoch_train = 0.0
    for batch_X, batch_y in loader_train:
        outputs = model(batch_X)
        loss_batch = criterion(outputs, batch_y.unsqueeze(1))
        loss_epoch_train += loss_batch

        optimizer.zero_grad()
        loss_batch.backward()
        optimizer.step()
    loss_all.append(loss_epoch_train)


  0%|          | 0/20 [00:00<?, ?it/s]

In [115]:
model.linear.weight

Parameter containing:
tensor([[-0.2188,  0.0287,  0.3830]], requires_grad=True)